In [113]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [71]:
# Temporary Only select the first file
data = pd.read_csv('./A03_data/training_data/order_data/order_data_2016-01-01', header=None)

In [72]:
# Load Data

files = glob.glob("A03_data/training_data/order_data/*")
# data = [pd.read_csv(f) for f in files]
# data = pd.concat(data, ignore_index=True)
data = data.iloc[:, 0].str.split("\t", expand=True)
data.columns = [
    "order_id",
    "driver_id",
    "passenger_id",
    "start_region_hash",
    "dest_region_hash",
    "Price",
    "Time",
]

region = pd.read_csv("A03_data/training_data/cluster_map/cluster_map", header=None)
region = region.iloc[:, 0].str.split("\t", expand=True)
region.columns = ["region_hash", "region_id"]

In [73]:
region

,region_hash,region_id
0,90c5a34f06ac86aee0fd70e2adce7d8a,1
1,f2c8c4bb99e6377d21de71275afd6cd2,2
2,58c7a4888306d8ff3a641d1c0feccbe3,3
3,b26a240205c852804ff8758628c0a86a,4
4,4b9e4cf2fbdc8281b8a1f9f12b80ce4d,5
...,...,...
61,a735449c5c09df639c35a7d61fad3ee5,62
62,0a5fef95db34383403d11cb6af937309,63
63,bf44d327f0232325c6d5280926d7b37d,64
64,825a21aa308dea206adb49c4b77c7805,65


In [74]:
data

,order_id,driver_id,passenger_id,start_region_hash,dest_region_hash,Price,Time
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24,2016-01-01 13:37:23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2,2016-01-01 09:47:54
2,abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9,2016-01-01 18:24:02
3,cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11,2016-01-01 22:13:27
4,139d492189ae5a933122c098f63252b3,NULL,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4,2016-01-01 17:00:06
...,...,...,...,...,...,...,...
501282,3f89a682ad12708095da4cbe2ccdab90,ac8c9e9a93fd7466ea26e729ce719dc4,56242551b109c1836958fa1ba09d94ba,74c1c25f4b283fa74a5514307b0d0278,74c1c25f4b283fa74a5514307b0d0278,7,2016-01-01 20:43:07
501283,677a78060b26e60191ff874b2b07be1d,48f1dca979de77f9a1db47cf867317f7,f5636ff3c2dbce0bf29283abbd0d48d7,2407d482f0ffa22a947068f2551fe62c,91690261186ae5bee8f83808ea1e4a01,7,2016-01-01 15:27:42
501284,b63daeda6d2f6d07af9d3bb37ebc8318,8c4c2fea7256337d2bbcc9fd2d1ae349,92ce1d70618e510792555ff10d96dd61,929ec6c160e6f52c20a4217c7978f681,d4ec2125aff74eded207d2d915ef682f,15,2016-01-01 21:49:58
501285,c0060c34335286a268f6783bf860062c,9ea6d19f8607f1a0df3fa1577b32953a,b030de3d2c88ba6d88b1eca06c74050f,825a21aa308dea206adb49c4b77c7805,2308e5edf2fbee52203b5c262f557ddf,12,2016-01-01 13:05:37


In [102]:
region_ids = region["region_id"]

region_list = [region for region in region_ids for _ in range(144)]
time_slots = list(range(0, 144)) * len(region_ids)

df = pd.DataFrame({"region_id": region_list, "time_slot": time_slots, "orders_count": 0, "unfulfilled_orders_count": 0, "gap": 0})

df

,region_id,time_slot,orders_count,unfulfilled_orders_count,gap
0,1,0,0,0,0
1,1,1,0,0,0
2,1,2,0,0,0
3,1,3,0,0,0
4,1,4,0,0,0
...,...,...,...,...,...
9499,66,139,0,0,0
9500,66,140,0,0,0
9501,66,141,0,0,0
9502,66,142,0,0,0


In [105]:
# Temporary
data = data.head(10000)

In [111]:
# iterate over data
for index, row in data.iterrows():

    # Match the start region hash in data to the region hash in region
    start_region_id = region[region["region_hash"] == row["start_region_hash"]]["region_id"].values[0]

    # Calculate the time slot
    _ = row["Time"].split(" ")[1]
    time_slot = (int(_.split(":")[0]) * 60 + int(_.split(":")[1])) // 10

    # Find the index of the row in df that matches the conditions
    df_index = df[(df["region_id"] == start_region_id) & (df["time_slot"] == time_slot)].index[0]

    # Increment the orders count
    if row["driver_id"] == "NULL":
        df.at[df_index, "unfulfilled_orders_count"] += 1
    else:
        df.at[df_index, "orders_count"] += 1

In [134]:
df["gap"] = df["orders_count"] - df["unfulfilled_orders_count"]
df

,region_id,time_slot,orders_count,unfulfilled_orders_count,gap
0,1,0,6,0,6
1,1,1,2,0,2
2,1,2,2,2,0
3,1,3,3,0,3
4,1,4,5,0,5
...,...,...,...,...,...
9499,66,139,0,0,0
9500,66,140,0,0,0
9501,66,141,0,0,0
9502,66,142,0,0,0


In [141]:
# Model Training
X = df[["region_id", "time_slot"]]  # input data
y = df["gap"]  # output data

# Convert 'time_slot' from string to datetime and then to timestamp
X.loc[:, "time_slot"] = (pd.to_datetime(X["time_slot"]).astype(int) / 10**9).astype(float)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

/tmp/ipykernel_486495/2574810115.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00e+00 1.00e-09 2.00e-09 ... 1.41e-07 1.42e-07 1.43e-07]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "time_slot"] = (pd.to_datetime(X["time_slot"]).astype(int) / 10**9).astype(float)


LinearRegression()

In [140]:
model.predict([[1, 1]])

/home/huzaifa/Desktop/University Desktop 6/App AI/Assignments/03/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([9608250.99741546])